In [1]:
import os
import json
from tqdm import tqdm
import shutil

def make_folders(path="output"):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    return path


def convert_coco_json_to_yolo_txt(output_path, json_file):

    path = make_folders(output_path)

    with open(json_file) as f:
        json_data = json.load(f)

    # write _darknet.labels, which holds names of all classes (one class per line)
    # label_file = os.path.join(output_path, "_darknet.labels")
    # with open(label_file, "w") as f:
    #     for category in tqdm(json_data["categories"], desc="Categories"):
    #         category_name = category["name"]
    #         f.write(f"{category_name}\n")

    for image in tqdm(json_data["images"], desc="Annotation txt for each iamge"):
        img_id = image["id"]
        img_name = image["file_name"]
        img_width = image["width"]
        img_height = image["height"]

        anno_in_image = [anno for anno in json_data["annotations"] if anno["image_id"] == img_id]
        anno_txt = os.path.join(output_path, img_name.split(".")[0] + ".txt")
        with open(anno_txt, "w") as f:
            for anno in anno_in_image:
                category = anno["category_id"]
                bbox_COCO = anno["bbox"]
                x, y, w, h = convert_bbox_coco2yolo(img_width, img_height, bbox_COCO)
                f.write(f"{category} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

    print("Converting COCO Json to YOLO txt finished!")

In [2]:
def convert_bbox_coco2yolo(img_width, img_height, bbox):
    """
    Convert bounding box from COCO  format to YOLO format

    Parameters
    ----------
    img_width : int
        width of image
    img_height : int
        height of image
    bbox : list[int]
        bounding box annotation in COCO format: 
        [top left x position, top left y position, width, height]

    Returns
    -------
    list[float]
        bounding box annotation in YOLO format: 
        [x_center_rel, y_center_rel, width_rel, height_rel]
    """
    
    # YOLO bounding box format: [x_center, y_center, width, height]
    # (float values relative to width and height of image)
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]

In [4]:
output_path = "./House/labels/train"
folder_path = "./House/labels/tl_json"
label_path = "./house.json"

output_path = "./House/labels/val"
folder_path = "./House/labels/vl_json"
label_path = "./house.json"

with open(label_path) as f:
    json_label = json.load(f)

print(json_label)

# path = make_folders(output_path)
for (dirpath, dirnames, filenames) in os.walk(folder_path):
    for filename in tqdm(filenames):
        
        anno_txt = os.path.join(output_path, filename.split(".")[0] + ".txt")
        with open(f'{folder_path}/{filename}', 'r') as file:
            data = json.load(file)
            anno_in_image = data['annotations']['bbox']
            with open(anno_txt, "w") as f:
                for anno in anno_in_image:
                    category = json_label[anno["label"]]
                    bbox_COCO = [anno['x'], anno['y'], anno['w'], anno['h']]
                    # print(category, bbox_COCO)
                    x, y, w, h = convert_bbox_coco2yolo(1280, 1280, bbox_COCO)
                    f.write(f"{category} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")
            # break

# print('f', f)

{'집전체': 0, '지붕': 1, '집벽': 2, '문': 3, '창문': 4, '굴뚝': 5, '연기': 6, '울타리': 7, '길': 8, '연못': 9, '산': 10, '나무': 11, '꽃': 12, '잔디': 13, '태양': 14}


100%|██████████| 1/1 [00:00<00:00, 1047.01it/s]


In [46]:
# Integrate for total model
# output_path = "./House/labels/train_total"
# folder_path = "./House/labels/tl_json"
# label_path = "./House/house.json"

output_path = "./HTP/labels/train"
folder_path = "./Tree/labels/tl_json"
label_path = "./Tree/tree.json"
int_start = 15

output_path = "./HTP/labels/val"
folder_path = "./PersonFemale/labels/vl_json"
label_path = "./PersonFemale/personfemale.json"
int_start = 29

output_path = "./HTP/labels/train"
folder_path = "./PersonMale/labels/tl_json"
label_path = "./PersonMale/personmale.json"
int_start = 29

with open(label_path) as f:
    json_label = json.load(f)

print(json_label)

# path = make_folders(output_path)
for (dirpath, dirnames, filenames) in os.walk(folder_path):
    for filename in tqdm(filenames):
        
        anno_txt = os.path.join(output_path, filename.split(".")[0] + ".txt")
        with open(f'{folder_path}/{filename}', 'r') as file:
            data = json.load(file)
            anno_in_image = data['annotations']['bbox']
            with open(anno_txt, "w") as f:
                for anno in anno_in_image:
                    category = json_label[anno["label"]]
                    bbox_COCO = [anno['x'], anno['y'], anno['w'], anno['h']]
                    # print(category, bbox_COCO)
                    x, y, w, h = convert_bbox_coco2yolo(1280, 1280, bbox_COCO)
                    f.write(f"{category+int_start} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")
            # break

{'사람전체': 0, '머리': 1, '얼굴': 2, '눈': 3, '코': 4, '입': 5, '귀': 6, '머리카락': 7, '목': 8, '상체': 9, '팔': 10, '손': 11, '다리': 12, '발': 13, '단추': 14, '주머니': 15, '운동화': 16, '남자구두': 17}


100%|██████████| 11200/11200 [00:03<00:00, 3332.41it/s]


In [49]:
# Example Request

import json
import requests

apiId = ""
apiKey = ""
# Run inference on an image
url = f"https://api.ultralytics.com/v1/predict/{apiId}"
headers = {"x-api-key": f"{apiKey}"}
data = {"size": 640, "confidence": 0.25, "iou": 0.45}
with open("~/Downloads/ex1.jpeg", "rb") as f:
	response = requests.post(url, headers=headers, data=data, files={"image": f})

# Check for successful response
response.raise_for_status()

# Print inference results
print(json.dumps(response.json(), indent=2))

{
  "data": [
    {
      "box": {
        "x1": 113.33133697509766,
        "x2": 277.315673828125,
        "y1": 31.125812530517578,
        "y2": 186.86239624023438
      },
      "class": 0,
      "confidence": 0.9368409514427185,
      "name": "\uc9d1\uc804\uccb4"
    },
    {
      "box": {
        "x1": 134.73887634277344,
        "x2": 181.14723205566406,
        "y1": 116.27368927001953,
        "y2": 184.94003295898438
      },
      "class": 3,
      "confidence": 0.9217495918273926,
      "name": "\ubb38"
    },
    {
      "box": {
        "x1": 114.72856140136719,
        "x2": 276.9072570800781,
        "y1": 49.34427261352539,
        "y2": 97.29202270507812
      },
      "class": 1,
      "confidence": 0.9142363667488098,
      "name": "\uc9c0\ubd95"
    },
    {
      "box": {
        "x1": 291.6980285644531,
        "x2": 399.9840087890625,
        "y1": 0.0,
        "y2": 282.6246337890625
      },
      "class": 11,
      "confidence": 0.9123105406761169,
      "n